In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch

from utils.model import extract_metrics

[2025-09-12 14:04:15,214] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/anaconda3/envs/diffusion/bin/../lib/gcc/x86_64-conda-linux-gnu/13.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/anaconda3/envs/diffusion/bin/../lib/gcc/x86_64-conda-linux-gnu/13.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/anaconda3/envs/diffusion/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-09-12 14:04:16,371] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [2]:
# from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


# def extract_metrics(log_dir: str) -> dict:
#     """
#     Extracts the latest scalar metrics from TensorBoard logs.

#     This function reads the TensorBoard event files in the specified log directory
#     and extracts the latest value for each scalar metric.

#     Args:
#         log_dir (str): Path to the directory containing TensorBoard event files.

#     Returns:
#         dict: A dictionary where keys are metric tags and values are the latest scalar values.
#     """
#     accumulator = EventAccumulator(log_dir)
#     accumulator.Reload()
#     metrics = {}
#     for tag in accumulator.Tags()["scalars"]:
#         events = accumulator.Scalars(tag)
#         # print(events)
#         if events:
#             metrics[tag] = events[-1].value
#     return metrics

In [3]:
ORDER = [
    "metric_gyr_pearson_X/test",
    "metric_gyr_pearson_Y/test",
    "metric_gyr_pearson_Z/test",
    "metric_gyr_pearson_norm/test",
    "metric_gyr_simVector_X/test",
    "metric_gyr_simVector_Y/test",
    "metric_gyr_simVector_Z/test",
    "metric_gyr_simVector_norm/test",
    "metric_gyr_simVector/test",
    # "metric_mse_gyr_X/test",
    # "metric_mse_gyr_Y/test",
    # "metric_mse_gyr_Z/test",
    # "metric_mse_gyr/test",
    "metric_naive_Angular_error_X/test",
    "metric_naive_Angular_error_Y/test",
    "metric_naive_Angular_error_Z/test",
    "metric_naive_Angular_error/test",
    #
    "metric_acc_pearson_X/test",
    "metric_acc_pearson_Y/test",
    "metric_acc_pearson_Z/test",
    "metric_acc_pearson_norm/test",
    "metric_acc_simVector_X/test",
    "metric_acc_simVector_Y/test",
    "metric_acc_simVector_Z/test",
    "metric_acc_simVector_norm/test",
    "metric_acc_simVector/test",
    # "metric_mse_acc_X/test",
    # "metric_mse_acc_Y/test",
    # "metric_mse_acc_Z/test",
    # "metric_mse_acc/test",
    "metric_naive_distance_error_X/test",
    "metric_naive_distance_error_Y/test",
    "metric_naive_distance_error_Z/test",
    "metric_naive_distance_error/test",
]

In [4]:
baseline_logs = Path("logs/diffusion_OIOD/baseline")
target_logs = Path("logs/diffusion_OIOD/version_0")

In [5]:
baseline_logs_metrics = extract_metrics(str(baseline_logs))
target_logs_matrics = extract_metrics(str(target_logs))

In [6]:
# create df with column name baseline and result
data = []
for key, value in baseline_logs_metrics.items():
    # Assuming value is a list of tuples and we want the first tuple's second element

    if key.replace("_mean", "") not in ORDER:
        continue

    baseline_val = value[0][1] if value and len(value[0]) > 1 else None
    target_val = (
        target_logs_matrics.get(key, [(None, None)])[0][1]
        if target_logs_matrics.get(key) and len(target_logs_matrics[key][0]) > 1
        else None
    )
    std_key = key.replace("_mean", "_std")
    baseline_std = (
        baseline_logs_metrics.get(std_key, [(None, None)])[0][1]
        if baseline_logs_metrics.get(std_key)
        and len(baseline_logs_metrics[std_key][0]) > 1
        else None
    )
    target_std = (
        target_logs_matrics.get(std_key, [(None, None)])[0][1]
        if target_logs_matrics.get(std_key) and len(target_logs_matrics[std_key][0]) > 1
        else None
    )
    print(
        f"{key}: {baseline_val} -> {target_logs_matrics.get(key, [(None, None)])[0][1]}"
    )
    data.append(
        {
            "metric": key.replace("_mean", ""),
            "baseline": baseline_val,
            "baseline_std": baseline_std,
            "result": target_val,
            "result_std": target_std,
        }
    )

df = pd.DataFrame.from_dict(data)
print(df.head(100))

metric_acc_pearson_X/test: 0.7213637828826904 -> 0.9755808711051941
metric_acc_pearson_Y/test: 0.46427881717681885 -> 0.9771765470504761
metric_acc_pearson_Z/test: -0.1268456131219864 -> 0.6259234547615051
metric_acc_pearson_norm/test: 0.5844262838363647 -> 0.9417152404785156
metric_acc_simVector/test: 0.3721754252910614 -> 0.8182361125946045
metric_acc_simVector_X/test: 0.6823089122772217 -> 0.9447945356369019
metric_acc_simVector_Y/test: 0.5231196284294128 -> 0.9317184686660767
metric_acc_simVector_Z/test: -0.08890233188867569 -> 0.5781954526901245
metric_acc_simVector_norm/test: 0.9635938405990601 -> 0.9824695587158203
metric_naive_distance_error_X/test: 0.21232858300209045 -> 0.034206829965114594
metric_naive_distance_error_Y/test: 0.2745972275733948 -> 0.041199345141649246
metric_naive_distance_error_Z/test: 0.25316575169563293 -> 0.010370557196438313
metric_naive_distance_error/test: 0.6400293707847595 -> 0.11229421198368073
metric_gyr_pearson_X/test: -0.055152177810668945 -> 0.1

In [7]:
# add the column of GAIN in df, which is the difference of the result column minus the baseline columns.
# df["GAIN"] = df["result"] - df["baseline"]
df = df.set_index("metric").loc[ORDER].reset_index()
print(df.head(100))

                                metric  baseline  baseline_std    result  \
0            metric_gyr_pearson_X/test -0.055152     -0.055152  0.104509   
1            metric_gyr_pearson_Y/test -0.091208     -0.091208  0.249037   
2            metric_gyr_pearson_Z/test  0.750473      0.750473  0.980660   
3         metric_gyr_pearson_norm/test  0.842328      0.842328  0.841285   
4          metric_gyr_simVector_X/test -0.153444     -0.153444  0.508807   
5          metric_gyr_simVector_Y/test -0.072373     -0.072373  0.499964   
6          metric_gyr_simVector_Z/test  0.808479      0.808479  0.948039   
7       metric_gyr_simVector_norm/test  0.985970      0.985970  0.982466   
8            metric_gyr_simVector/test  0.194221      0.194221  0.652270   
9    metric_naive_Angular_error_X/test  0.096126      0.096126  0.022963   
10   metric_naive_Angular_error_Y/test  0.087656      0.087656  0.015787   
11   metric_naive_Angular_error_Z/test  0.130152      0.130152  0.019432   
12     metri

In [8]:
# Positive correlation
POS = ["pearson", "simVector"]
NEG = ["mse", "naive"]

In [9]:
# convert the df to latex format and for the GAIN column if the row name contains the keyword in POS means the positive correlation, so positive should be red and negative should be green. On the contrary, if the key contains the NEG any keywords then it means the negative relationship, where the negative should be red and positive should be green
def generate_aligned_latex_table(df, pos_keywords, neg_keywords):
    """
    Process a DataFrame by:
      1. Coloring the 'GAIN' column based on the relationship found in the 'metric' column.
      2. Combining baseline and result with their respective std columns, unless std is NaN.
      3. Formatting all numeric columns (except 'GAIN') with .4f format.
      4. Converting the DataFrame to a LaTeX table.
      5. Post-processing the LaTeX string to align columns for easier editing.

    Returns:
      A well-aligned LaTeX table string.
    """

    def color_gain(row):
        metric = row["metric"]
        gain = row["GAIN"]
        is_pos = any(p in metric for p in pos_keywords)
        is_neg = any(n in metric for n in neg_keywords)
        if is_pos:
            return (
                r"\textcolor{red}{%.2f}" % gain
                if gain > 0
                else r"\textcolor{green}{%.2f}" % gain
            )
        elif is_neg:
            return (
                r"\textcolor{red}{%.2f}" % gain
                if gain < 0
                else r"\textcolor{green}{%.2f}" % gain
            )
        else:
            return "%.2f" % gain

    def format_with_std(value, std):
        if pd.isna(std):
            return f"{value:.2f}"
        else:
            return f"{value:.2f} $\pm$ {std:.2f}"

    def align_latex_table(latex_str):
        lines = latex_str.splitlines()
        table_rows = []
        for idx, line in enumerate(lines):
            if "&" in line and line.strip().endswith(r"\\"):
                parts = line.rstrip().rstrip(r"\\").split(" & ")
                table_rows.append((idx, parts))

        if not table_rows:
            return latex_str

        num_cols = len(table_rows[0][1])
        col_widths = [0] * num_cols
        for _, parts in table_rows:
            for i, cell in enumerate(parts):
                col_widths[i] = max(col_widths[i], len(cell))

        for idx, parts in table_rows:
            padded_parts = [cell.ljust(col_widths[i]) for i, cell in enumerate(parts)]
            lines[idx] = " & ".join(padded_parts) + r" \\"

        return "\n".join(lines)

    df_copy = df.copy()

    # Format baseline and result with std, skipping ± if std is NaN
    # df_copy["baseline"] = df_copy.apply(
    #     lambda row: format_with_std(row["baseline"], row["baseline_std"]), axis=1
    # )
    # df_copy["result"] = df_copy.apply(
    #     lambda row: format_with_std(row["result"], row["result_std"]), axis=1
    # )

    # Apply color formatting to GAIN

    # Drop std columns
    df_copy.drop(columns=["baseline_std", "result_std"], inplace=True)
    for col in ["baseline", "result"]:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].apply(
                lambda x: round(x, 2) if pd.notna(x) else np.nan
            )

    df_copy["GAIN"] = df_copy["result"] - df_copy["baseline"]
    df_copy["GAIN"] = df_copy.apply(color_gain, axis=1)
    # Format baseline and result to .2f
    for col in ["baseline", "result"]:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].apply(
                lambda x: f"{x:.2f}" if pd.notna(x) else ""
            )

    # Format other numeric columns
    for col in df_copy.columns:
        if col not in ["GAIN", "baseline", "result"] and pd.api.types.is_numeric_dtype(
            df_copy[col]
        ):
            df_copy[col] = df_copy[col].apply(
                lambda x: f"{x:.2f}" if pd.notna(x) else ""
            )

    latex_str = df_copy.to_latex(index=False, escape=False)
    aligned_latex_str = align_latex_table(latex_str)

    return aligned_latex_str


# Get the aligned LaTeX table string:
latex_table = generate_aligned_latex_table(df, POS, NEG)
print(latex_table)

\begin{tabular}{llll}
\toprule
metric                             & baseline & result & GAIN                      \\
\midrule
metric_gyr_pearson_X/test          & -0.06    & 0.10   & \textcolor{red}{0.16}     \\
metric_gyr_pearson_Y/test          & -0.09    & 0.25   & \textcolor{red}{0.34}     \\
metric_gyr_pearson_Z/test          & 0.75     & 0.98   & \textcolor{red}{0.23}     \\
metric_gyr_pearson_norm/test       & 0.84     & 0.84   & \textcolor{green}{0.00}   \\
metric_gyr_simVector_X/test        & -0.15    & 0.51   & \textcolor{red}{0.66}     \\
metric_gyr_simVector_Y/test        & -0.07    & 0.50   & \textcolor{red}{0.57}     \\
metric_gyr_simVector_Z/test        & 0.81     & 0.95   & \textcolor{red}{0.14}     \\
metric_gyr_simVector_norm/test     & 0.99     & 0.98   & \textcolor{green}{-0.01}  \\
metric_gyr_simVector/test          & 0.19     & 0.65   & \textcolor{red}{0.46}     \\
metric_naive_Angular_error_X/test  & 0.10     & 0.02   & \textcolor{red}{-0.08}    \\
metric_naive_A

In [10]:
# # convert the df to latex format and for the GAIN column if the row name contains the keyword in POS means the positive correlation, so positive should be red and negative should be green. On the contrary, if the key contains the NEG any keywords then it means the negative relationship, where the negative should be red and positive should be green
# def color_gain(row):
#     metric = row["metric"]
#     gain = row["GAIN"]
#     is_pos = any(p in metric for p in POS)
#     is_neg = any(n in metric for n in NEG)
#     if is_pos:
#         # Positive correlation: positive gain is red, negative is green
#         if gain > 0:
#             return r"\textcolor{red}{%.4f}" % gain
#         else:
#             return r"\textcolor{green}{%.4f}" % gain
#     elif is_neg:
#         # Negative correlation: negative gain is red, positive is green
#         if gain < 0:
#             return r"\textcolor{red}{%.4f}" % gain
#         else:
#             return r"\textcolor{green}{%.4f}" % gain
#     else:
#         return "%.4f" % gain

# df_latex = df.copy()
# df_latex["GAIN"] = df_latex.apply(color_gain, axis=1)
# latex_str = df_latex.to_latex(index=False, escape=False)
# print(latex_str)